In [7]:

import cv2
import numpy as np
import os
from skimage.feature.texture import graycomatrix, graycoprops
import pandas as pd
import json
import math

def rgb_to_hsi(img):

    with np.errstate(divide='ignore', invalid='ignore'):

        #Load image with 32 bit floats as variable type
        bgr = np.float32(img)/255

        #Separate color channels
        blue = bgr[:,:,0]
        green = bgr[:,:,1]
        red = bgr[:,:,2]

        #Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        #Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        #Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                        ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        #Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

# Function to extract features from a single image
def extract_features(image):
        # If there is no alpha channel, process the entire image
        #if zero dont input
        if np.any(image!=[0,0,0]):
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Normalize RGB values to [0, 1] for accurate HSV conversion
            rgb_normalized = rgb.astype(np.float32) / 255.0
    
            # Convert RGB to HSV
            hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)
    
            # Convert to grayscale for GLCM
            gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
            # Convert RGB to HSI
            hsi = rgb_to_hsi(image)

            # RGB
            r, g, b = np.mean(rgb[:,:,0]), np.mean(rgb[:,:,1]), np.mean(rgb[:,:,2])
        
            # HSV
            h_hsv, s_hsv, v_hsv = np.mean(hsv[:,:,0]), np.mean(hsv[:,:,1]), np.mean(hsv[:,:,2])
        
            # HSI (HLS approximation)
            h_hsi, s_hsi, i_hsi = np.mean(hsi[:,:,0]), np.mean(hsi[:,:,1]), np.mean(hsi[:,:,2])
        
            # GLCM
            glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                 symmetric=True, normed=True)
            contrast = graycoprops(glcm, 'contrast')[0, 0]
            correlation = graycoprops(glcm, 'correlation')[0, 0]
            energy = graycoprops(glcm, 'energy')[0, 0]
            homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        
            return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
        else: 
            return None, None, None, None

        
# Directory containing the images
directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\toLoad"  
# Replace "path/to/images" with the actual path to your image directory

# List to store features for all images
all_features = []

# Iterate over images in the directory
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load the image
        image = cv2.imread(os.path.join(directory, filename), cv2.IMREAD_UNCHANGED)
        
        # Extract features
        r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity = extract_features(image)
        
        # Determine grading based on filename
        if filename.startswith("grade_A"):
            grading = "A"
        elif filename.startswith("grade_B"):
            grading = "B"
        elif filename.startswith("grade_C"):
            grading = "C"
        else:
            grading = "Unknown"
        
            
        all_features.append({
            'grading': grading,
            'R': r, 
            'G': g,
            'B': b,
            'H_HSV': h_hsv,
            'S_HSV': s_hsv,
            'V_HSV': v_hsv,
            'H_HSI': h_hsi,
            'S_HSI': s_hsi,
            'I_HSI': i_hsi,
            'CONTRAST': contrast,
            'CORRELATION': correlation,
            'ENERGY': energy,
            'HOMOGENEITY': homogeneity
        })

# Convert all_features list to a DataFrame
df = pd.DataFrame(all_features)

# Save DataFrame to a CSV file
df.to_csv('tuna_image_features.csv', index=False)

print("csv file generated")

csv file generated


In [40]:
# importing libraries  
import numpy as nm  
import matplotlib.pyplot as mtp  
import pandas as pd  
  
#importing datasets  
data_set= pd.read_csv('test.csv')  
  
#Extracting Independent and dependent Variable  
x= data_set.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13]].values  
y= data_set.iloc[:, 0].values  
  
# Splitting the dataset into training and test set.  
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.20, random_state=42)  
  
#feature Scaling  
from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test) 

#Fitting K-NN classifier to the training set  
from sklearn.neighbors import KNeighborsClassifier  
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )  
classifier.fit(x_train, y_train)  

#Predicting the test set result  
y_pred= classifier.predict(x_test)

#Creating the Confusion matrix  
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib 

# Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Additional Model Evaluation Metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Step 7: Save the trained model and the scaler
joblib.dump(classifier, 'knn_model.pkl')
joblib.dump(st_x, 'scaler.pkl')


Accuracy: 0.934931506849315

Classification Report:
              precision    recall  f1-score   support

           A       0.96      0.91      0.93        95
           B       0.91      0.93      0.92        91
           C       0.94      0.96      0.95       106

    accuracy                           0.93       292
   macro avg       0.94      0.93      0.93       292
weighted avg       0.94      0.93      0.93       292


Confusion Matrix:
[[ 86   4   5]
 [  4  85   2]
 [  0   4 102]]


['scaler.pkl']

In [8]:
#input image
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math

def rgb_to_hsi(img):

    with np.errstate(divide='ignore', invalid='ignore'):

        #Load image with 32 bit floats as variable type
        bgr = np.float32(img)/255

        #Separate color channels
        blue = bgr[:,:,0]
        green = bgr[:,:,1]
        red = bgr[:,:,2]

        #Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        #Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        #Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                        ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        #Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
        #if zero dont input
        if np.any(image!=[0,0,0]):
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Normalize RGB values to [0, 1] for accurate HSV conversion
            rgb_normalized = rgb.astype(np.float32) / 255.0
    
            # Convert RGB to HSV
            hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)
    
            # Convert to grayscale for GLCM
            gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
            # Convert RGB to HSI
            hsi = rgb_to_hsi(image)

            # RGB
            r, g, b = np.mean(rgb[:,:,0]), np.mean(rgb[:,:,1]), np.mean(rgb[:,:,2])
        
            # HSV
            h_hsv, s_hsv, v_hsv = np.mean(hsv[:,:,0]), np.mean(hsv[:,:,1]), np.mean(hsv[:,:,2])
        
            # HSI (HLS approximation)
            h_hsi, s_hsi, i_hsi = np.mean(hsi[:,:,0]), np.mean(hsi[:,:,1]), np.mean(hsi[:,:,2])
        
            # GLCM
            glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                 symmetric=True, normed=True)
            contrast = graycoprops(glcm, 'contrast')[0, 0]
            correlation = graycoprops(glcm, 'correlation')[0, 0]
            energy = graycoprops(glcm, 'energy')[0, 0]
            homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        
            return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
        else: 
            return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

     # Apply background subtraction
    fg_mask = bg_subtractor.apply(image)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # Draw bounding box around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Zoom in to a specific area inside the bounding box
        zoom_factor = 3  # Adjust the zoom factor as needed
        center_x = x + w // 2
        center_y = y + h // 2
        zoom_size = min(w, h) // zoom_factor
        x_zoom = max(0, center_x - zoom_size // 2)
        y_zoom = max(0, center_y - zoom_size // 2)
        zoomed_image = image[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

        # Resize the zoomed image to 100x100 pixels
        zoomed_resized = cv2.resize(zoomed_image, (32, 32))

        # Extract features from the zoomed-in frame with foreground mask
        zoomed_fg_mask = fg_mask[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()
        features = extract_features(cv2.bitwise_and(zoomed_image, zoomed_fg_mask))
    
    # Extract features
    features = extract_features(zoomed_resized)
    
    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])
        
        # Predict the grade
        grade = knn_model.predict(scaled_features)

        image = cv2.resize(image,(700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)
        
        # Draw the text on the zoomed image
        cv2.putText(image, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
        # Display the frame with bounding box
        cv2.imshow('Frame with Bounding Box', image)

        zoomed_resized = cv2.resize(zoomed_resized, (700, 700))
        # Draw the outline of the text on the zoomed image
        outline_thickness = 2
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)

        # Draw the text on the zoomed image
        cv2.putText(zoomed_resized, "Predicted Grade: {}".format(grade[0]), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the zoomed image with predicted grade
        cv2.imshow('Zoomed In with Predicted Grade', zoomed_resized)
    
        # Wait for a key press to close the windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to open file explorer for picking an image
def open_file_explorer():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Open file explorer and return the selected file path
    file_path = filedialog.askopenfilename()

    if file_path:
        # If a file is selected, classify the image
        predicted_grade = classify_image(file_path)
        print("Predicted Grade:", predicted_grade)
    else:
        print("No file selected.")

# Call the function to open file explorer
open_file_explorer()


Predicted Grade: A


In [2]:
#input image
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog
import math

def rgb_to_hsi(img):

    with np.errstate(divide='ignore', invalid='ignore'):

        #Load image with 32 bit floats as variable type
        bgr = np.float32(img)/255

        #Separate color channels
        blue = bgr[:,:,0]
        green = bgr[:,:,1]
        red = bgr[:,:,2]

        #Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        #Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        #Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                        ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        #Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

def extract_features(image):
        #if zero dont input
        if np.any(image!=[0,0,0]):
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Normalize RGB values to [0, 1] for accurate HSV conversion
            rgb_normalized = rgb.astype(np.float32) / 255.0
    
            # Convert RGB to HSV
            hsv = cv2.cvtColor(rgb_normalized, cv2.COLOR_RGB2HSV)
    
            # Convert to grayscale for GLCM
            gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
            # Convert RGB to HSI
            hsi = rgb_to_hsi(image)

            # RGB
            r, g, b = np.mean(rgb[:,:,0]), np.mean(rgb[:,:,1]), np.mean(rgb[:,:,2])
        
            # HSV
            h_hsv, s_hsv, v_hsv = np.mean(hsv[:,:,0]), np.mean(hsv[:,:,1]), np.mean(hsv[:,:,2])
        
            # HSI (HLS approximation)
            h_hsi, s_hsi, i_hsi = np.mean(hsi[:,:,0]), np.mean(hsi[:,:,1]), np.mean(hsi[:,:,2])
        
            # GLCM
            glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                 symmetric=True, normed=True)
            contrast = graycoprops(glcm, 'contrast')[0, 0]
            correlation = graycoprops(glcm, 'correlation')[0, 0]
            energy = graycoprops(glcm, 'energy')[0, 0]
            homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        
            return r, g, b, h_hsv, s_hsv, v_hsv, h_hsi, s_hsi, i_hsi, contrast, correlation, energy, homogeneity
        else: 
            return None, None, None, None

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Function to classify an image
def classify_image(img):
    global _grade
    # Extract features
    features = extract_features(img)
    
    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])
        
        # Predict the grade
        grade = knn_model.predict(scaled_features)
        # print(grade)
        _grade = grade
        
        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

def main():
    # Open the default camera (usually the built-in webcam)
    cap = cv2.VideoCapture(1)

    # Check if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    # Create a background subtractor object
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Check if the frame was captured successfully
        if not ret:
            print("Error: Failed to capture frame.")
            break

        # Apply background subtraction
        fg_mask = bg_subtractor.apply(frame)

        # Find contours in the foreground mask
        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the largest contour (if any)
        largest_contour = max(contours, key=cv2.contourArea, default=None)

        # Draw bounding box around the largest contour (if any)
        if largest_contour is not None:
            x, y, w, h = cv2.boundingRect(largest_contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Zoom in to a specific area inside the bounding box
            zoom_factor = 3  # Adjust the zoom factor as needed
            center_x = x + w // 2
            center_y = y + h // 2
            zoom_size = min(w, h) // zoom_factor
            x_zoom = max(0, center_x - zoom_size // 2)
            y_zoom = max(0, center_y - zoom_size // 2)
            zoomed_image = frame[y_zoom:y_zoom + zoom_size, x_zoom:x_zoom + zoom_size].copy()

            # Resize the zoomed image to 100x100 pixels
            zoomed_resized = cv2.resize(zoomed_image, (400, 400))
            # cv2.imshow('Zoomed Bounding Box', zoomed_resized)
            try:
                zoomed_resized = cv2.resize(zoomed_image, (32, 32))
                classify_image(zoomed_resized)
                # Draw the outline of the text on the zoomed image
                outline_thickness = 2
                cv2.putText(frame, "Predicted Grade: {}".format(_grade), (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), outline_thickness + 2)
                
                # Draw the text on the zoomed image
                cv2.putText(frame, "Predicted Grade: {}".format(_grade), (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                # print(f"grade = {_grade}")
            except:
                 pass

        # Display the frame with bounding box (if any)
        cv2.imshow('Largest Foreground Object', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close the windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Function to convert RGB to HSI
def rgb_to_hsi(image):
    r, g, b = cv2.split(image.astype(float) / 255.0)
    intensity = (r + g + b) / 3.0
    min_rgb = np.minimum(np.minimum(r, g), b)
    saturation = 1 - (3.0 / (r + g + b + 1e-10) * min_rgb)
    num = 0.5 * ((r - g) + (r - b))
    den = np.sqrt((r - g)**2 + (r - b) * (g - b)) + 1e-10
    hue = np.arccos(num / den)
    hue[b > g] = 2 * np.pi - hue[b > g]
    hue = np.degrees(hue)
    # Convert HSI components to uint8
    hue = np.uint8(hue)
    saturation = np.uint8(saturation * 255)
    intensity = np.uint8(intensity * 255)
    return cv2.merge([hue, saturation, intensity])


# Function to display images
def display_images(images, titles):
    num_images = len(images)
    num_titles = len(titles)
    
    if num_images != num_titles:
        print("Number of images does not match number of titles.")
        return
    
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    
    for i in range(num_images):
        axes[i].imshow(images[i])
        axes[i].set_title(titles[i])
        axes[i].axis('off')
    
    plt.show()

# Directory containing the images
directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\toLoad"  

# List to store images
images = []

# Iterate over images in the directory
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load the image
        image = cv2.imread(os.path.join(directory, filename), cv2.IMREAD_COLOR)
        
        # Convert RGB to HSV
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Convert RGB to HSI
        hsi_image = cv2.cvtColor(rgb_to_hsi(image), cv2.COLOR_HSV2RGB)
        
        # Store the images and their titles
        images.append(image)
        images.append(hsv_image)
        images.append(hsi_image)
        
        # Assuming 'rgb_image', 'hsv_image', and 'hsi_image' are your images
        images = [image, hsv_image, hsi_image]
        titles = ['RGB', 'HSV', 'HSI']
        
        # Display the images
        display_images(images, titles)



In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Function to extract dominant colors using k-means clustering
def get_dominant_colors(image, k=5):
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(image)
    colors = kmeans.cluster_centers_
    return colors

# Function to plot the color palette
def plot_color_palette(colors, grade, title):
    palette = np.zeros((50, 300, 3), dtype=np.uint8)
    steps = 300 // colors.shape[0]
    for i, color in enumerate(colors):
        palette[:, i * steps:(i + 1) * steps, :] = color
    plt.figure()
    plt.title(title)
    plt.imshow(palette)
    plt.axis('off')
    plt.show()

# Load the CSV file
df = pd.read_csv('tuna_image_features.csv')

# Assuming your CSV file contains strings that can be evaluated to lists
df['rgb'] = df['rgb'].apply(eval)
df['hsv'] = df['hsv'].apply(eval)
df['hsi'] = df['hsi'].apply(eval)

# Dictionary to store images for each grade
grade_images = {'A': [], 'B': [], 'C': []}

# Iterate over the DataFrame
for index, row in df.iterrows():
    grading = row['grading']
    rgb = np.array(row['rgb'])
    hsv = np.array(row['hsv'])
    hsi = np.array(row['hsi'])

    # Store the image in the corresponding grade list
    if grading in grade_images:
        grade_images[grading].append({'rgb': rgb, 'hsv': hsv, 'hsi': hsi})

# Generate and display color palettes for each grade
for grade, images in grade_images.items():
    if images:
        # Combine all images of the same grade into one array for each feature
        combined_rgb = np.vstack([image['rgb'] for image in images])
        combined_hsv = np.vstack([image['hsv'] for image in images])
        combined_hsi = np.vstack([image['hsi'] for image in images])

        # Get dominant colors for each feature
        dominant_colors_rgb = get_dominant_colors(combined_rgb)
        dominant_colors_hsv = get_dominant_colors(combined_hsv)
        dominant_colors_hsi = get_dominant_colors(combined_hsi)

        # Plot color palettes for each feature
        plot_color_palette(dominant_colors_rgb, grade, f"RGB Color Palette for Grade {grade}")
        plot_color_palette(dominant_colors_hsv, grade, f"HSV Color Palette for Grade {grade}")
        plot_color_palette(dominant_colors_hsi, grade, f"HSI Color Palette for Grade {grade}")
